In [2]:
import clip
import torch
from torch import nn
from tqdm import tqdm

In [3]:
detr_clip = torch.load('outputs/detr_clip_train0.pth')

In [5]:
sample = detr_clip[0]

In [6]:
list(sample.keys())

['detr_f', 'clip_f', 'outputs', 'targets']

In [7]:
sample['detr_f'].size(), sample['clip_f'].size(), list(sample['outputs'].keys()), list(sample['targets'].keys())

(torch.Size([100, 256]),
 torch.Size([1024]),
 ['pred_logits', 'pred_boxes'],
 ['boxes', 'labels', 'image_id', 'area', 'iscrowd', 'orig_size', 'size'])

In [ ]:
class DetrClipFuser(nn.Module):
    def __init__(self, d_detr, d_clip, text_encoded) -> None:
        super().__init__()
        self.d_model = d_clip
        self.linear_projection = nn.Linear(d_detr, d_clip)
        decoder_layer = nn.TransformerDecoderLayer(d_model=self.d_model, nhead=12)
        self.decoder = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=6)
    
    def forward(self, clip_img_feature, detr_proposals):
        detr_projected = self.linear_projection(detr_proposals)
        out = self.decoder(tgt=detr_projected, memory=clip_img_feature)